In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/articles.pql"
article_df = pd.read_pickle(data_path)

In [5]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/cs.pql"
data = None
cs_categories = pd.read_pickle(data_path)

In [6]:
cs_categories = cs_categories[cs_categories.Label == "Computer Science, Artificial Intelligence"]

In [7]:
data = None
data = cs_categories.merge(article_df, 
                            how = 'inner',
                            on = 'ArticleID')

In [8]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [9]:
data['title_without_punct'] = data['Title'].swifter.apply(lambda x: x.lower().translate(translator))

Pandas Apply: 100%|██████████| 159054/159054 [00:00<00:00, 210897.68it/s]


In [10]:
data["title_without_stopwords"] = data['title_without_punct'].swifter.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords and not word.isdigit() and len(word) >= 4]))

Pandas Apply: 100%|██████████| 159054/159054 [00:03<00:00, 45145.60it/s]


In [11]:
pd.set_option('display.max_colwidth', -1)

In [12]:
data[["Title","title_without_stopwords"]].head()

,Title,title_without_stopwords
0,TEST FEATURE-SELECTION FOR A COMPLEX ELECTROHYDRAULIC SERVO SYSTEM USING FREQUENCY-RESPONSE MEASUREMENTS,test feature selection complex electrohydraulic servo system using frequency response measurements
1,THEOREM-PROVING WITH ABSTRACTION,theorem proving abstraction
2,THE APPLICATION OF PARALLEL PROJECTIONS TO 3-DIMENSIONAL OBJECT LOCATION IN INDUSTRIAL ASSEMBLY,application parallel projections dimensional object location industrial assembly
3,ESTIMATES OF THE DYNAMIC CHARACTERISTICS OF EXPERIMENTAL RESEARCH AUTOMATIZATION MULTICOMPUTER SYSTEMS,estimates dynamic characteristics experimental research automatization multicomputer systems
4,SEARCH FOR AN OPTIMAL SOLUTION OF COMBINATORIAL PROBLEMS WITH BOOLEAN VARIABLES ON REARRANGEABLE CELLULAR COMPUTING ARRAYS,search optimal solution combinatorial problems boolean variables rearrangeable cellular computing arrays


In [13]:
data = data.sort_values(by="PubYear")

In [14]:
max(data.PubYear)

2016

In [15]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data.title_without_stopwords)

In [16]:
row_counts = data.groupby('PubYear').size().reset_index(name="count")

In [17]:
counts =  list(row_counts["count"])

__What are the overall topwords__

In [18]:
N = 50

In [19]:
vocab = vectorizer.vocabulary_
rev_vocab = {index:word for word,index in vocab.items()}

In [20]:
# Highest Frequency words all time
all_time_word_frequencies = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
word_index_with_freq = {index:freq for index,freq in enumerate(all_time_word_frequencies)}

In [21]:
word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}

In [22]:
sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)

In [23]:
top_words_all_corpus = pd.DataFrame(list(sorted_word_with_freq[:50]), columns=["word","frequency"])

In [24]:
datatoplot = [go.Line(
            x=top_words_all_corpus["word"],
            y=top_words_all_corpus["frequency"]
    )]



layout = go.Layout(
    title='Top Frequency Words',
    
    xaxis=dict(
        title='Word',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    )
)

fig1 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig1, filename='freq')

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [25]:
index_start = 0
yearly_top_words = []

N = 5000

for i in range(len(row_counts)):
    #print(i)
    offset = index_start + counts[i]
    tmp = vectorized_words[index_start:offset,:]
    yearly_word_freq = tmp.sum(axis=0).reshape(-1,).tolist()[0]
    
    word_index_with_freq = {index:freq for index,freq in enumerate(yearly_word_freq)}
    word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}
    sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)
    top_words = [word for (word,freq) in sorted_word_with_freq[:N]]
    yearly_top_words.append(top_words)

In [26]:
years = list(range(1945, 2017))
overlapping_word_counts = []
new_word_ratio = []

for i in range(1, len(row_counts)):
    
    overlap_count = len([word for word in yearly_top_words[i] if word in yearly_top_words[i-1]])
    new_word_count = N - overlap_count
    overlapping_word_counts.append(overlap_count)
    new_word_ratio.append(new_word_count/overlap_count)

__How many top words are overlapping with the words from adjacent year__

In [27]:
datatoplot = [go.Scatter(
            x=years,
            y=overlapping_word_counts,
            mode = 'lines+markers'
    )]



layout = go.Layout(
    title='Overlap',
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f',
        ),
        range=[1945, 2017]
    ),
    yaxis=dict(
        title='Overlap Count',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        ),
        range=[0, 5000]
    )
)

fig2 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig2, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [28]:
datatoplot = [go.Scatter(
            x=years,
            y=new_word_ratio,
            mode = 'lines+markers'
    )]



layout = go.Layout(
    title='New Word Ratio',
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Overlap Count',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        ),
        range=[0, 1]
    )
)

fig2 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig2, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



__Consider three past years and do the same analysis__


    - Top words during previous every three year window
    - Percentage of overlap of current year with the previous window

In [32]:
index_start = 0

window_size = 4
top_words_during_window = []

N = 5000


for i in range(len(row_counts)-window_size):
    offset = index_start + counts[i] + counts[i+1] + counts[i+2]
    tmp = vectorized_words[index_start:offset,:]
    yearly_word_freq = tmp.sum(axis=0).reshape(-1,).tolist()[0]
    
    word_index_with_freq = {index:freq for index,freq in enumerate(yearly_word_freq)}
    word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}
    sorted_word_with_freq = sorted(word_with_freq.items(), key=operator.itemgetter(1), reverse=True)
    top_words = [word for (word,freq) in sorted_word_with_freq[:N]]
    top_words_during_window.append(top_words)

In [33]:
overlapping_word_counts = []
new_word_ratio = []


for i in range(len(row_counts)- window_size):
    
    overlap_count = len([word for word in yearly_top_words[i] if word in top_words_during_window[i-1]])
    new_word_count = N - overlap_count
    overlapping_word_counts.append(overlap_count)
    new_word_ratio.append(new_word_count/overlap_count)

In [34]:
years = list(range(1948, 2016))

datatoplot = [go.Scatter(
            x=years,
            y=new_word_ratio,
            mode = 'lines+markers'
    )]



layout = go.Layout(
    title='New word ratio compared to top words in the past N years',
    
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Overlap Count',
        titlefont=dict(
            family='Lato',
            color='#7f7f7f'
        ),
        range=[0, 1]
    )
)

fig3 = go.Figure(data=datatoplot, layout=layout)
py.iplot(fig3, filename='freq')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

